### Module \#13 Assignment Solutions

**1. GUI Assignment**

In [ ]:

import tkinter as tk
from tkinter import messagebox
import random


class NumberGuesser:
    def __init__(self):
        self.guessed_list = []

    def add_guess(self, guess):
        self.guessed_list.append(guess)

    def reset_game(self):
        self.guessed_list = []


class GuessingGameApp(tk.Tk):
    MAX = 10  # You can adjust the maximum value here

    def __init__(self):
        super().__init__()
        self.title("Number Guessing Game")
        self.game = NumberGuesser()
        self.secret_number = None
        self.buttons = []
        self.start_button = tk.Button(self, text="Start Game", command=self.start_game)
        self.start_button.pack(pady=10)
        self.guess_frame = tk.Frame(self)
        self.guess_frame.pack()
        
        for number in range(1, self.MAX + 1):
            btn = tk.Button(self.guess_frame, text=str(number), command=lambda n=number: self.guess(n))
            btn.grid(row=(number-1)//5, column=(number-1)%5)
            self.buttons.append(btn)

    def start_game(self):
        self.secret_number = random.randint(1, self.MAX)
        self.game.reset_game()
        for btn in self.buttons:
            btn.config(state=tk.NORMAL)

    def guess(self, number):
        self.game.add_guess(number)
        if number == self.secret_number:
            messagebox.showinfo("Winner!", "You guessed correctly!")
        else:
            self.buttons[number - 1].config(state=tk.DISABLED)


app = GuessingGameApp()
app.mainloop()


**2. Database and GUI Assignment**

In [ ]:

import sqlite3
import tkinter as tk
from tkinter import simpledialog, messagebox, scrolledtext


# Database setup
def create_database():
    conn = sqlite3.connect('school.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS person (
            person_id INTEGER PRIMARY KEY AUTOINCREMENT,
            firstname TEXT NOT NULL,
            lastname TEXT NOT NULL
        );
    ''')
    c.execute('''
        CREATE TABLE IF NOT EXISTS student (
            student_id INTEGER PRIMARY KEY AUTOINCREMENT,
            person_id INTEGER NOT NULL,
            major TEXT NOT NULL,
            startdate TEXT NOT NULL,
            FOREIGN KEY(person_id) REFERENCES person(person_id)
        );
    ''')
    conn.commit()
    conn.close()
    messagebox.showinfo("Info", "Database and tables created successfully!")


def create_person(firstname, lastname):
    conn = sqlite3.connect('school.db')
    c = conn.cursor()
    c.execute('INSERT INTO person (firstname, lastname) VALUES (?, ?)', (firstname, lastname))
    conn.commit()
    conn.close()


def create_student(firstname, lastname, major, startdate):
    conn = sqlite3.connect('school.db')
    c = conn.cursor()
    c.execute('SELECT person_id FROM person WHERE firstname=? AND lastname=?', (firstname, lastname))
    person_id = c.fetchone()[0]
    c.execute('INSERT INTO student (person_id, major, startdate) VALUES (?, ?, ?)', (person_id, major, startdate))
    conn.commit()
    conn.close()


def view_table(table):
    conn = sqlite3.connect('school.db')
    c = conn.cursor()
    c.execute(f'SELECT * FROM {table}')
    rows = c.fetchall()
    conn.close()
    display_rows(rows)


def display_rows(rows):
    output.delete('1.0', tk.END)
    for row in rows:
        output.insert(tk.END, str(row) + "\n")


def add_person():
    if len(entry_firstname.get())==0 or len(entry_lastname.get()) == 0:
        messagebox.showinfo("Info", "No info to add.")
    else:
        create_person(entry_firstname.get(), entry_lastname.get())
        messagebox.showinfo("Info", "Person added successfully!")


def add_student():
    major = simpledialog.askstring("Input", "Enter student's major:")
    startdate = simpledialog.askstring("Input", "Enter start date (YYYY-MM-DD):")
    create_student(entry_firstname.get(), entry_lastname.get(), major, startdate)
    messagebox.showinfo("Info", "Student added successfully!")



# GUI setup.
root = tk.Tk()
root.title("School Database")
tk.Button(root, text="Create Database & Table", command=create_database).pack()
entry_firstname = tk.Entry(root)
entry_firstname.pack()
entry_lastname = tk.Entry(root)
entry_lastname.pack()
tk.Button(root, text="Add Person", command=add_person).pack()
tk.Button(root, text="Add Student", command=add_student).pack()
output = tk.Text(root, height=10, width=50)
output.pack()
tk.Button(root, text="View Person Table", command=lambda: view_table('person')).pack()
tk.Button(root, text="View Student Table", command=lambda: view_table('student')).pack()
tk.Button(root, text="Exit", command=root.quit).pack()
root.mainloop()
